# <center>Statistical Associations Analysis

## Context

This notebook is part of the **Under-Five Mortality Modelling Project**, which uses data from the **2022 Demographic and Health Surveys (DHS)** to identify factors influencing child survival outcomes.  
Before moving into predictive or explanatory modeling, it is essential to understand how individual features—both **continuous** (e.g., birth weight, maternal age) and **categorical** (e.g., region, maternal education, household wealth) are associated with under-five mortality.

Understanding these relationships provides a solid statistical foundation for building, validating, and interpreting models that predict or explain child mortality risk.

---

## Purpose and Objectives

The **Statistical Associations** notebook aims to:
- Examine the **relationship** between predictor variables and mortality outcomes (neonatal, infant, or under-five).
- Identify variables that show **statistically significant differences** between survival groups.
- Quantify the **strength of associations** through appropriate effect size metrics.
- Provide a **theoretical and empirical basis** for variable inclusion in the modeling notebook.

This analysis acts as a bridge between **Exploratory Data Analysis (EDA)** and **Model Building**, ensuring that the modeling process is data-driven and statistically justified.

---

## Structure and Layout

1. **Introduction & Data Import**  
   - Loads the cleaned dataset and sets up analysis functions.

2. **Continuous Variables vs. Mortality**  
   - Applies non-parametric tests (e.g., *Mann–Whitney U*) to compare continuous features between children who survived and those who did not.  
   - Computes effect sizes to assess the magnitude of differences.

3. **Categorical Variables vs. Mortality**  
   - Uses Chi-square or Fisher’s exact tests to detect significant associations between categorical predictors and mortality.  
   - Visualizes results using bar charts or heatmaps for easier interpretation.

4. **Interpretation and Discussion**  
   - Summarizes which variables are significantly associated with mortality.  
   - Highlights patterns consistent with literature on child survival determinants.

5. **Summary of Key Associations**  
   - Provides a concise table of p-values and effect sizes to inform feature selection for modeling.

---

### Importance and Expected Impact

Conducting statistical association tests at this stage contributes to the project in several ways:

- **Feature Selection Insight**  
  Identifies the most relevant and informative variables to include in regression or machine learning models.

- **Model Transparency and Credibility**  
  Establishes a clear, evidence-based rationale for modeling choices.

- **Data Quality and Reliability Check**  
  Detects non-informative or inconsistent variables that could distort model performance.

- **Policy-Relevant Evidence**  
  Offers interpretable, data-driven insights into socioeconomic and demographic disparities in under-five mortality, supporting targeted interventions.

- **Strengthening Research Validity**  
  Ensures that subsequent modeling is grounded in sound statistical reasoning rather than purely algorithmic exploration.

---

-  In summary, this notebook serves as the **analytical backbone** connecting descriptive exploration with predictive modeling.  
-  It ensures that all subsequent model-building steps are **statistically informed**, **interpretable**, and **aligned with public health objectives** to reduce under-five mortality.



In [19]:
# Import relevant libraries
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

# Effect size for categorical variables
import scipy.stats as ss

# Load data
df = pd.read_csv("u5mr_clean.csv")

# Check target distribution
df['under5_mortality'].value_counts(normalize=True) * 100

under5_mortality
0    96.446493
1     3.553507
Name: proportion, dtype: float64

In [20]:
# Continuous Variables vs. Mortality

continuous_vars = [
    'Birth weight in kilograms (3 decimals)',
    'Childs weight in kilograms (1 decimal)',
    'Childs height in centimeters (1 decimal)',
    'Months of breastfeeding',
    'When child put to breast',
    'Height/Age standard deviation (new WHO)',
    'Weight/Age standard deviation (new WHO)',
    'Weight/Height standard deviation (new WHO)'
]  

results_cont = []

for var in continuous_vars:
    alive = df[df["under5_mortality"] == 0][var].dropna()
    dead = df[df["under5_mortality"] == 1][var].dropna()

    # Default test
    test_used = "Mann-Whitney U"
    stat, pval = stats.mannwhitneyu(alive, dead)  # fallback

    # Run normality check only if enough variation
    if alive.nunique() > 10 and dead.nunique() > 10:
        _, p_norm_alive = stats.shapiro(alive.sample(min(len(alive), 500)))
        _, p_norm_dead = stats.shapiro(dead.sample(min(len(dead), 500)))

        if p_norm_alive > 0.05 and p_norm_dead > 0.05:
            stat, pval = stats.ttest_ind(alive, dead, equal_var=False)
            test_used = "t-test"

    results_cont.append([
        var, test_used, round(stat, 3), round(pval, 4)
    ])

results_df = pd.DataFrame(
    results_cont, columns=["Variable", "Test", "Statistic", "p-value"]
)

results_df

,Variable,Test,Statistic,p-value
0,Birth weight in kilograms (3 decimals),Mann-Whitney U,6396584.0,0.2384
1,Childs weight in kilograms (1 decimal),Mann-Whitney U,6548237.0,0.9336
2,Childs height in centimeters (1 decimal),Mann-Whitney U,6543032.0,0.9620
3,Months of breastfeeding,Mann-Whitney U,8069257.0,0.0000
4,When child put to breast,Mann-Whitney U,6806595.5,0.0006
5,Height/Age standard deviation (new WHO),Mann-Whitney U,6528111.0,0.9563
6,Weight/Age standard deviation (new WHO),Mann-Whitney U,6517354.0,0.8977
7,Weight/Height standard deviation (new WHO),Mann-Whitney U,6535051.0,0.9943


## Interpretation of Continuous Variables vs Mortality

1. Birth weight (p = 0.2384) - **Not statistically significant.**    
Birth weight did not differ meaningfully between children who survived and those who died under-5 in this dataset.

2. Child’s weight (p = 0.9336) - **Not significant.**    
No evidence that current weight is associated with under-5 mortality.

3. Child’s height (p = 0.9620) - **Not significant.**    
No measurable difference in height between groups.

4. Months of breastfeeding (p < 0.0001) - **Highly significant.**    
Duration of breastfeeding strongly differs between children who survived and those who died before age 5.  
Survivors likely had longer breastfeeding periods, consistent with protective effects.  

5. When child was put to breast (p = 0.0006) - **Significant.**  
Timing of initiating breastfeeding is associated with survival.  
Early initiation seems protective, while delays are associated with higher mortality risk.

6. Height-for-age Z-score (p = 0.9563) - **Not significant.**  
Stunting did not show a direct statistical difference between survivors and non-survivors here.

7. Weight-for-age Z-score (p = 0.8977) - **Not significant.**      
Underweight status was not directly different between groups.

8. Weight-for-height Z-score (p = 0.9943) - **Not significant.**  
Wasting did not differ between mortality groups.

**Summary**  
- Key determinants identified so far:
    - Months of breastfeeding and timing of breastfeeding initiation are significantly associated with under-5 mortality.   
 
- Other anthropometric and weight/height-related measures do not show significant differences in this dataset (at least not in univariate comparisons).

In [21]:
# Effect Size for Continuous Variables
continuous_vars = [
    'Birth weight in kilograms (3 decimals)',
    'Childs weight in kilograms (1 decimal)',
    'Childs height in centimeters (1 decimal)',
    'Months of breastfeeding',
    'When child put to breast',
    'Height/Age standard deviation (new WHO)',
    'Weight/Age standard deviation (new WHO)',
    'Weight/Height standard deviation (new WHO)'
]  

results_cont = []

for var in continuous_vars:
    alive = df[df["under5_mortality"] == 0][var].dropna()
    dead = df[df["under5_mortality"] == 1][var].dropna()
    
    # Normality check (sample to 500 to avoid issues)
    _, p_norm_alive = stats.shapiro(alive.sample(min(len(alive), 500), random_state=42))
    _, p_norm_dead = stats.shapiro(dead.sample(min(len(dead), 500), random_state=42))
    
    if p_norm_alive > 0.05 and p_norm_dead > 0.05:
        # Independent t-test
        stat, pval = stats.ttest_ind(alive, dead)
        test_used = "t-test"
        
        # Effect size: Cohen's d
        pooled_sd = np.sqrt(((alive.std() ** 2) + (dead.std() ** 2)) / 2)
        cohens_d = (alive.mean() - dead.mean()) / pooled_sd
        effect_size = cohens_d
        effect_label = "Cohen's d"
        
    else:
        # Mann-Whitney U test
        stat, pval = stats.mannwhitneyu(alive, dead, alternative="two-sided")
        test_used = "Mann-Whitney U"
        
        # Effect size: rank-biserial correlation
        n1, n2 = len(alive), len(dead)
        rbc = 1 - (2 * stat) / (n1 * n2)
        effect_size = rbc
        effect_label = "Rank-biserial r"
    
    results_cont.append([var, test_used, round(stat, 3), round(pval, 4), effect_label, round(effect_size, 3)])

pd.DataFrame(results_cont, columns=["Variable", "Test", "Statistic", "p-value", "Effect Size", "Value"])


c:\Users\pc\anaconda4\envs\anc-env\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


,Variable,Test,Statistic,p-value,Effect Size,Value
0,Birth weight in kilograms (3 decimals),Mann-Whitney U,6396584.0,0.2384,Rank-biserial r,0.021
1,Childs weight in kilograms (1 decimal),Mann-Whitney U,6548237.0,0.9336,Rank-biserial r,-0.002
2,Childs height in centimeters (1 decimal),Mann-Whitney U,6543032.0,0.9620,Rank-biserial r,-0.001
3,Months of breastfeeding,Mann-Whitney U,8069257.0,0.0000,Rank-biserial r,-0.235
4,When child put to breast,Mann-Whitney U,6806595.5,0.0006,Rank-biserial r,-0.041
5,Height/Age standard deviation (new WHO),Mann-Whitney U,6528111.0,0.9563,Rank-biserial r,0.001
6,Weight/Age standard deviation (new WHO),Mann-Whitney U,6517354.0,0.8977,Rank-biserial r,0.003
7,Weight/Height standard deviation (new WHO),Mann-Whitney U,6535051.0,0.9943,Rank-biserial r,0.000


`Understanding the Metrics`

- Test: Mann-Whitney U compares distributions of a continuous variable across two groups (e.g., died vs. survived).

- p-value: probability that any observed difference is due to chance (α=0.05 threshold).

-  Effect Size (rank-biserial r): quantifies the magnitude of difference on a scale from –1 to 1.

    - |r| < 0.1 negligible

    - 0.1 ≤ |r| < 0.3 small

    - 0.3 ≤ |r| < 0.5 moderate

    - |r| ≥ 0.5 large

Detailed Variable Interpretation

1. Birth weight (kg)
    - p = 0.2384 (not significant)
    - r = 0.021 (negligible) → No meaningful difference in birth weight between children who died and those who survived.

2. Child’s weight (kg)
    - p = 0.9336
    - r = –0.002 (negligible) → Distributions virtually identical across mortality outcomes.

3. Child’s height (cm)
    - p = 0.9620

    - r = –0.001 (negligible) → No detectable association with under-5 death.

4. Months of breastfeeding
    - p < 0.0001 (highly significant)
    - r = –0.235 (small-to-moderate) → Children who survived tended to be breastfed longer. The negative r suggests the “died” group had shorter breastfeeding duration.

5. When put to breast (time to initiation)
    - p = 0.0006
    - r = –0.041 (small) → Earlier initiation of breastfeeding is modestly associated with survival.

6. Height-for-age z-score (WHO)
    - p = 0.9563
    - r = 0.001 (negligible) → Stunting (as measured here) does not differ by mortality in this sample.

7. Weight-for-age z-score (WHO)
    - p = 0.8977
    - r = 0.003 (negligible) → Underweight status shows no group difference.

8. Weight-for-height z-score (WHO)
    - p = 0.9943
    - r = 0.000 (negligible) → Wasting is unrelated to mortality here.

**Practical Takeaways**

- Breastfeeding practices emerge as the only continuous predictors with both statistical and practical relevance.

- Anthropometry at the time of survey doesn’t distinguish survivors from non-survivors—possibly because measurements occur post-mortality for the deceased and don’t reflect conditions at risk time.

In [22]:
# Categorical Variables vs Mortality

categorical_vars = [
    'Region',
    'Age in 5-year groups',
    'Type of place of residence',
    'Highest educational level',
    'Religion',
    'Ethnicity',
    'Wealth index combined',
    'Type of cooking fuel (smoke exposure, indoor air pollution)',
    'Sex of child',
    'Place of delivery',
    'Size of child at birth',
    'Has health card and or other vaccination document',
    'Received BCG',
    'Received POLIO 0',
    'Received POLIO 1',
    'Received POLIO 2',
    'Received POLIO 3',
    'Received MEASLES 1',
    'Received MEASLES 2',
    'Received inactivated polio (IPV)',
    'Received Pentavalent 1',
    'Received Pentavalent 2',
    'Received Pentavalent 3',
    'Received Pneumococcal 1',
    'Received Pneumococcal 2',
    'Received Pneumococcal 3',
    'Received Rotavirus 1',
    'Received Rotavirus 2',
    'Place where most vaccinations were received',
    'Yellow fever vaccine',
    'Currently breastfeeding',
    'Given child anything other than breast milk',
    'In contact with someone with cough or TB',
    'Source of drinking water',
    'Main floor material',
    'Visited health facility last 12 months',
    'Getting medical help for self: distance to health facility',
    'Mode of transportation to nearest healthcare facility',
    'prenatal_help',
    'delivery_help',
    'child_death_history']  

results_cat = []

for var in categorical_vars:
    ct = pd.crosstab(df[var], df["under5_mortality"])
    chi2, p, dof, ex = stats.chi2_contingency(ct)
    results_cat.append([var, "Chi-square", round(chi2, 3), round(p, 4)])

pd.DataFrame(results_cat, columns=["Variable", "Test", "Chi2", "p-value"])


,Variable,Test,Chi2,p-value
0,Region,Chi-square,68.029,0.0190
1,Age in 5-year groups,Chi-square,16.501,0.0113
2,Type of place of residence,Chi-square,0.942,0.3318
3,Highest educational level,Chi-square,17.755,0.0005
4,Religion,Chi-square,9.613,0.3828
5,Ethnicity,Chi-square,22.751,0.0299
6,Wealth index combined,Chi-square,1.467,0.8324
7,"Type of cooking fuel (smoke exposure, indoor a...",Chi-square,11.847,0.5402
8,Sex of child,Chi-square,1.159,0.2816
9,Place of delivery,Chi-square,31.475,0.0005


### Significant associations (p < 0.05)

- Region (p = 0.019) → mortality differs by geographic region.

- Age in 5-year groups (p = 0.011) → maternal age group is linked to neonatal mortality.

- Education (p = 0.0005) → higher maternal education levels are associated with lower mortality.

- Ethnicity (p = 0.0299) → some ethnic groups show different mortality risks.

- Place of delivery (p = 0.0005) → delivering at a facility vs. home strongly matters.

- Size at birth (p < 0.001) → very strong association (small babies have higher mortality).

- Vaccination & documentation variables (all p < 0.001) → overwhelmingly strong associations; children who received vaccines or had a vaccination card were far less likely to die neonatally.

- Breastfeeding (p < 0.001) → breastfeeding status strongly linked to survival.

- Feeding other than breast milk (p = 0.0003) → introducing other foods/liquids early is associated with mortality.

- TB/cough exposure (p = 0.003) → environmental exposure to illness increases risk.

- Prenatal help (p = 0.0165) → whether the mother received skilled help during pregnancy is associated with survival.

- Yellow fever vaccine (p < 0.001) → associated, but might be confounded (since coverage is limited to some regions).

### Non-significant associations (p ≥ 0.05)

- Residence (urban/rural), wealth index, religion, fuel, sex of child, source of water, floor material, mode of transportation, distance to facility, delivery help (who assisted) → no evidence of significant differences in neonatal mortality.

- These could still be important predictors in multivariable models (sometimes they interact with other variables), but individually they don’t show strong associations here.

In [23]:
# Effect Size for Categorical Variables (Cramér's V)

def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

effect_sizes = []
for var in categorical_vars:
    ct = pd.crosstab(df[var], df["under5_mortality"])
    effect_sizes.append([var, round(cramers_v(ct), 3)])

pd.DataFrame(effect_sizes, columns=["Variable", "Cramer's V"]).sort_values("Cramer's V", ascending=False)


,Variable,Cramer's V
40,child_death_history,0.468
28,Place where most vaccinations were received,0.142
17,Received MEASLES 1,0.138
19,Received inactivated polio (IPV),0.119
13,Received POLIO 0,0.110
25,Received Pneumococcal 3,0.109
16,Received POLIO 3,0.109
22,Received Pentavalent 3,0.108
27,Received Rotavirus 2,0.107
15,Received POLIO 2,0.102


**Interpretation of Categorical Effect Sizes**
- Only vaccination-related variables show small associations (Cramer’s V ≈ 0.10–0.15) with under-5 mortality.

- All other categorical factors—including socioeconomic, demographic, and environmental measures—have negligible associations (Cramer’s V < 0.10).

`Cramer’s V Thresholds`

- Less than 0.10 – negligible association

- 0.10 to 0.20 – small association

- 0.20 to 0.40 – moderate association

- Above 0.40 – strong association

**Detailed Variable Insights**

- Place where most vaccinations were received (V = 0.142) Small effect suggests location of immunization has modest linkage to survival.

- Received MEASLES 1 (V = 0.138) Early measles vaccination shows a small but meaningful association with lower mortality.

- Received IPV (inactivated polio vaccine) (V = 0.119) Small association hints at protective impact of polio immunization.

- POLIO 0, POLIO 3, Pneumococcal 3, Pentavalent 3 (V ≈ 0.109–0.110) Third-dose completion for key vaccines all register small effects.

- Pentavalent 2, Rotavirus 1/2, Pneumococcal 2 (V ≈ 0.101–0.102) Second-dose coverage also shows small associations, consistent across multiple antigens.

- Has health card/document (V = 0.097) Marginally under the small-effect threshold but still indicates that documented immunization tracking relates to survival.

- Other variables (size at birth, region, maternal education, wealth, water source, fuel type, etc.) all have V < 0.10, indicating no meaningful categorical association with under-5 mortality in this dataset.

In [ ]:
# Combine Continuous + Categorical Results

# Continuous (already computed earlier)
results_cont = []
for var in continuous_vars:
    alive = df[df["under5_mortality"] == 0][var].dropna()
    dead = df[df["under5_mortality"] == 1][var].dropna()
    
    # Normality check
    _, p_norm_alive = stats.shapiro(alive.sample(min(len(alive), 500), random_state=42))
    _, p_norm_dead = stats.shapiro(dead.sample(min(len(dead), 500), random_state=42))
    
    if p_norm_alive > 0.05 and p_norm_dead > 0.05:
        stat, pval = stats.ttest_ind(alive, dead)
        test_used = "t-test"
        pooled_sd = np.sqrt(((alive.std() ** 2) + (dead.std() ** 2)) / 2)
        effect_size = (alive.mean() - dead.mean()) / pooled_sd
        effect_label = "Cohen's d"
    else:
        stat, pval = stats.mannwhitneyu(alive, dead, alternative="two-sided")
        test_used = "Mann-Whitney U"
        n1, n2 = len(alive), len(dead)
        effect_size = 1 - (2 * stat) / (n1 * n2)  # Rank-biserial correlation
        effect_label = "Rank-biserial r"
    
    # Interpret effect size
    if effect_label == "Cohen's d":
        if abs(effect_size) < 0.2: interpretation = "negligible"
        elif abs(effect_size) < 0.5: interpretation = "small"
        elif abs(effect_size) < 0.8: interpretation = "medium"
        else: interpretation = "large"
    else:  # rank-biserial r
        if abs(effect_size) < 0.1: interpretation = "negligible"
        elif abs(effect_size) < 0.3: interpretation = "small"
        elif abs(effect_size) < 0.5: interpretation = "medium"
        else: interpretation = "large"
    
    results_cont.append([
        var, "Continuous", test_used, round(stat, 3), round(pval, 4),
        effect_label, round(effect_size, 3), interpretation
    ])

df_cont_summary = pd.DataFrame(
    results_cont,
    columns=["Variable", "Type", "Test", "Statistic", "p-value", "Effect Size", "Value", "Interpretation"]
)


# Categorical (already computed earlier)
results_cat = []
for var in categorical_vars:
    ct = pd.crosstab(df[var], df["under5_mortality"])
    chi2, p, dof, ex = stats.chi2_contingency(ct)
    effect_size = cramers_v(ct)
    
    # Interpret Cramér's V (common cutoffs)
    if effect_size < 0.1: interpretation = "negligible"
    elif effect_size < 0.3: interpretation = "small"
    elif effect_size < 0.5: interpretation = "medium"
    else: interpretation = "large"
    
    results_cat.append([
        var, "Categorical", "Chi-square", round(chi2, 3), round(p, 4),
        "Cramér's V", round(effect_size, 3), interpretation
    ])

df_cat_summary = pd.DataFrame(
    results_cat,
    columns=["Variable", "Type", "Test", "Statistic", "p-value", "Effect Size", "Value", "Interpretation"]
)


# ---------------------------
# Final Combined Summary Table
# ---------------------------
summary_table = pd.concat([df_cont_summary, df_cat_summary], ignore_index=True)

# Sort by strongest effect size first
summary_table = summary_table.sort_values("Value", ascending=False).reset_index(drop=True)

summary_table


c:\Users\pc\anaconda4\envs\anc-env\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


,Variable,Type,Test,Statistic,p-value,Effect Size,Value,Interpretation
0,child_death_history,Categorical,Chi-square,4286.838,0.0000,Cramér's V,0.468,medium
1,Place where most vaccinations were received,Categorical,Chi-square,407.752,0.0000,Cramér's V,0.142,small
2,Received MEASLES 1,Categorical,Chi-square,377.137,0.0000,Cramér's V,0.138,small
3,Received inactivated polio (IPV),Categorical,Chi-square,282.798,0.0000,Cramér's V,0.119,small
4,Received POLIO 0,Categorical,Chi-square,238.385,0.0000,Cramér's V,0.110,small
5,Received Pneumococcal 3,Categorical,Chi-square,235.391,0.0000,Cramér's V,0.109,small
6,Received POLIO 3,Categorical,Chi-square,236.690,0.0000,Cramér's V,0.109,small
7,Received Pentavalent 3,Categorical,Chi-square,230.484,0.0000,Cramér's V,0.108,small
8,Received Rotavirus 2,Categorical,Chi-square,229.201,0.0000,Cramér's V,0.107,small
9,Received POLIO 2,Categorical,Chi-square,206.140,0.0000,Cramér's V,0.102,small


In [ ]:
# Add "Selected for Modeling" Flag

def flag_selection(pval, interpretation):
    if pval < 0.05 and interpretation != "negligible":
        return "Yes"
    else:
        return "No"

# Apply flag to the summary table
summary_table["Selected for Modeling"] = summary_table.apply(
    lambda row: flag_selection(row["p-value"], row["Interpretation"]), axis=1
)

# Reorder columns for readability
summary_table = summary_table[
    ["Variable", "Type", "Test", "Statistic", "p-value", 
     "Effect Size", "Value", "Interpretation", "Selected for Modeling"]
]

summary_table


,Variable,Type,Test,Statistic,p-value,Effect Size,Value,Interpretation,Selected for Modeling
0,child_death_history,Categorical,Chi-square,4286.838,0.0000,Cramér's V,0.468,medium,Yes
1,Place where most vaccinations were received,Categorical,Chi-square,407.752,0.0000,Cramér's V,0.142,small,Yes
2,Received MEASLES 1,Categorical,Chi-square,377.137,0.0000,Cramér's V,0.138,small,Yes
3,Received inactivated polio (IPV),Categorical,Chi-square,282.798,0.0000,Cramér's V,0.119,small,Yes
4,Received POLIO 0,Categorical,Chi-square,238.385,0.0000,Cramér's V,0.110,small,Yes
5,Received Pneumococcal 3,Categorical,Chi-square,235.391,0.0000,Cramér's V,0.109,small,Yes
6,Received POLIO 3,Categorical,Chi-square,236.690,0.0000,Cramér's V,0.109,small,Yes
7,Received Pentavalent 3,Categorical,Chi-square,230.484,0.0000,Cramér's V,0.108,small,Yes
8,Received Rotavirus 2,Categorical,Chi-square,229.201,0.0000,Cramér's V,0.107,small,Yes
9,Received POLIO 2,Categorical,Chi-square,206.140,0.0000,Cramér's V,0.102,small,Yes


In [ ]:
from scipy.stats import pointbiserialr

int_vars = [
    'Number of household members (listed)',
    'Birth order number',
    'Preceding birth interval (months)',
    'Succeeding birth interval (months)',
    'Duration of pregnancy in months',
    'Timing of 1st antenatal check (months)',
    'Number of antenatal visits during pregnancy',
    'Entries in pregnancy and postnatal care roster',
    'Minutes to nearest healthcare facility',
    'Number of tetanus injections before birth',
    'Number of tetanus injections before pregnancy'
]

def analyze_discrete_vars(df, discrete_vars, target='mortality'):
    """
    Analyze association of discrete numeric variables with a binary target (mortality).
    Computes point-biserial correlation and Cohen's d.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataset containing variables.
    discrete_vars : list
        List of discrete numeric variable names.
    target : str
        Name of the binary outcome variable (default = 'mortality').
    
    Returns
    -------
    results : pd.DataFrame
        DataFrame with variable name, correlation, p-value, Cohen's d, and notes.
    """
    results = []

    # Vars better interpreted as categorical/ordinal
    flagged = [
        'Sons who have died',
        'Daughters who have died',
        'Birth order number',
        'Age at death (months, imputed)'
    ]

    for var in discrete_vars:
        x = df[var].dropna()
        y = df.loc[x.index, target]

        note = ""
        if var in flagged:
            note = "Consider treating as categorical/ordinal"

        try:
            # Point-biserial correlation
            corr, pval = pointbiserialr(y, x)

            # Cohen's d between mortality groups
            group0 = x[y == 0]
            group1 = x[y == 1]
            pooled_sd = np.sqrt(((group0.std() ** 2) + (group1.std() ** 2)) / 2)
            cohens_d = (group0.mean() - group1.mean()) / pooled_sd if pooled_sd > 0 else np.nan

            results.append({
                "Variable": var,
                "Correlation (r_pb)": round(corr, 3),
                "p-value": round(pval, 4),
                "Cohen's d": round(cohens_d, 3),
                "Notes": note
            })
        except Exception as e:
            results.append({
                "Variable": var,
                "Correlation (r_pb)": np.nan,
                "p-value": np.nan,
                "Cohen's d": np.nan,
                "Notes": f"Failed: {e}"
            })

    return pd.DataFrame(results)

# Example use:
discrete_results = analyze_discrete_vars(df, int_vars, target='under5_mortality')
discrete_results

,Variable,Correlation (r_pb),p-value,Cohen's d,Notes
0,Number of household members (listed),-0.051,0.0000,0.273,
1,Birth order number,0.010,0.1800,-0.049,Consider treating as categorical/ordinal
2,Preceding birth interval (months),0.003,0.6962,-0.014,
3,Succeeding birth interval (months),-0.087,0.0000,0.398,
4,Duration of pregnancy in months,-0.124,0.0000,0.441,
5,Timing of 1st antenatal check (months),-0.022,0.0022,0.133,
6,Number of antenatal visits during pregnancy,-0.015,0.0414,0.080,
7,Entries in pregnancy and postnatal care roster,0.070,0.0000,-0.344,
8,Minutes to nearest healthcare facility,0.002,0.7961,-0.010,
9,Number of tetanus injections before birth,-0.008,0.2772,0.044,


In [32]:
# ==========================
# Association Summary Table
# ==========================

# --- Helper function for Cramér's V ---
def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# -----------------------------
# Continuous Variables
# -----------------------------
results_cont = []
for var in continuous_vars:
    alive = df[df["under5_mortality"] == 0][var].dropna()
    dead = df[df["under5_mortality"] == 1][var].dropna()
    
    # Normality check
    _, p_norm_alive = stats.shapiro(alive.sample(min(len(alive), 500), random_state=42))
    _, p_norm_dead = stats.shapiro(dead.sample(min(len(dead), 500), random_state=42))
    
    if p_norm_alive > 0.05 and p_norm_dead > 0.05:
        stat, pval = stats.ttest_ind(alive, dead)
        test_used = "t-test"
        pooled_sd = np.sqrt(((alive.std() ** 2) + (dead.std() ** 2)) / 2)
        effect_size = (alive.mean() - dead.mean()) / pooled_sd
        effect_label = "Cohen's d"
    else:
        stat, pval = stats.mannwhitneyu(alive, dead, alternative="two-sided")
        test_used = "Mann-Whitney U"
        n1, n2 = len(alive), len(dead)
        effect_size = 1 - (2 * stat) / (n1 * n2)  # Rank-biserial correlation
        effect_label = "Rank-biserial r"
    
    # Interpret effect size
    if effect_label == "Cohen's d":
        if abs(effect_size) < 0.2: interpretation = "negligible"
        elif abs(effect_size) < 0.5: interpretation = "small"
        elif abs(effect_size) < 0.8: interpretation = "medium"
        else: interpretation = "large"
    else:  # rank-biserial r
        if abs(effect_size) < 0.1: interpretation = "negligible"
        elif abs(effect_size) < 0.3: interpretation = "small"
        elif abs(effect_size) < 0.5: interpretation = "medium"
        else: interpretation = "large"
    
    results_cont.append([
        var, "Continuous", test_used, round(stat, 3), round(pval, 4),
        effect_label, round(effect_size, 3), interpretation
    ])

df_cont_summary = pd.DataFrame(
    results_cont,
    columns=["Variable", "Type", "Test", "Statistic", "p-value",
             "Effect Size", "Value", "Interpretation"]
)


# -----------------------------
# Categorical Variables
# -----------------------------
results_cat = []
for var in categorical_vars:
    ct = pd.crosstab(df[var], df["under5_mortality"])
    chi2, p, dof, ex = stats.chi2_contingency(ct)
    effect_size = cramers_v(ct)
    
    # Interpret Cramér's V
    if effect_size < 0.1: interpretation = "negligible"
    elif effect_size < 0.3: interpretation = "small"
    elif effect_size < 0.5: interpretation = "medium"
    else: interpretation = "large"
    
    results_cat.append([
        var, "Categorical", "Chi-square", round(chi2, 3), round(p, 4),
        "Cramér's V", round(effect_size, 3), interpretation
    ])

df_cat_summary = pd.DataFrame(
    results_cat,
    columns=["Variable", "Type", "Test", "Statistic", "p-value",
             "Effect Size", "Value", "Interpretation"]
)


# -----------------------------
# Discrete Numeric Variables
# -----------------------------
results_disc = []
for var in int_vars:
    x = df[var].dropna()
    y = df.loc[x.index, "under5_mortality"]
    
    corr, pval = stats.pointbiserialr(y, x)
    
    # Interpret correlation
    if abs(corr) < 0.1: interpretation = "negligible"
    elif abs(corr) < 0.3: interpretation = "small"
    elif abs(corr) < 0.5: interpretation = "medium"
    else: interpretation = "large"
    
    results_disc.append([
        var, "Discrete numeric", "Point-biserial r", round(corr, 3), round(pval, 4),
        "Correlation r", round(corr, 3), interpretation
    ])

df_disc_summary = pd.DataFrame(
    results_disc,
    columns=["Variable", "Type", "Test", "Statistic", "p-value",
             "Effect Size", "Value", "Interpretation"]
)


# -----------------------------
# Final Combined Summary
# -----------------------------
summary_table = pd.concat([df_cont_summary, df_cat_summary, df_disc_summary], ignore_index=True)

#Sort by p-value first (ascending), then by effect size (descending for stronger effects)
summary_table = summary_table.sort_values(
    by=["p-value", "Value"], 
    ascending=[True, False]
).reset_index(drop=True)

summary_table


c:\Users\pc\anaconda4\envs\anc-env\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


,Variable,Type,Test,Statistic,p-value,Effect Size,Value,Interpretation
0,child_death_history,Categorical,Chi-square,4286.838,0.0000,Cramér's V,0.468,medium
1,Place where most vaccinations were received,Categorical,Chi-square,407.752,0.0000,Cramér's V,0.142,small
2,Received MEASLES 1,Categorical,Chi-square,377.137,0.0000,Cramér's V,0.138,small
3,Received inactivated polio (IPV),Categorical,Chi-square,282.798,0.0000,Cramér's V,0.119,small
4,Received POLIO 0,Categorical,Chi-square,238.385,0.0000,Cramér's V,0.110,small
5,Received POLIO 3,Categorical,Chi-square,236.690,0.0000,Cramér's V,0.109,small
6,Received Pneumococcal 3,Categorical,Chi-square,235.391,0.0000,Cramér's V,0.109,small
7,Received Pentavalent 3,Categorical,Chi-square,230.484,0.0000,Cramér's V,0.108,small
8,Received Rotavirus 2,Categorical,Chi-square,229.201,0.0000,Cramér's V,0.107,small
9,Received POLIO 2,Categorical,Chi-square,206.140,0.0000,Cramér's V,0.102,small
